In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def check_dicom(filename):
    """
    This function reads in a .dcm file, checks the important fields for our device, 
    and returns a numpy array of just the imaging data.
    """
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    
    if ds.Modality != 'DX':
        print("There is an error in the file - {}. Incorrect image type.".format(filename))
        return None
    if ds.PatientPosition not in ['AP', 'PA']:
        print("There is an error in the file - {}. Incorrect patient position.".format(filename))
        return None
    if ds.BodyPartExamined != 'CHEST':
        print("There is an error in the file - {}. Incorrect body part examined.".format(filename))
        return None
    img = ds.pixel_array
    return img
     
def preprocess_image(img,img_mean,img_std,img_size): 
    """
    This function takes the numpy array output by check_dicom and
    runs the appropriate pre-processing needed for our model input.
    
    """

    proc_img = (img - img_mean)/img_std
    proc_img =  np.resize(proc_img, img_size)
    
    return proc_img
 
def load_model(model_path, weight_path):
    """This function loads in our trained model w/ weights and compiles it. """
    with open(model_path, 'r') as json_file:
        saved_model = json_file.read()
        
    model = tf.keras.models.model_from_json(saved_model)
    model.load_weights(weight_path)
    
    return model

def predict_image(model, img, thresh): 
    """
    This function uses our device's threshold parameters to predict whether or not
    the image shows the presence of pneumonia using our trained model.
    
    """    
    result = model.predict(img)  
    predict=result[0]
    prediction='No pneumonia'
    if(predict>thresh):
        prediction='Pneumonia'
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'final_model.json' # Path to saved model
weight_path = 'xray_class_model.best.hdf5' # Path to saved best weights

IMG_SIZE=(1, 224,224,3) # This might be different if you did not use vgg16

mean_list = []
std_list = []
for i in test_dicoms:
    ds = pydicom.dcmread('test2.dcm') 
    img = ds.pixel_array
    mean = np.mean(img)/255
    stand_dev = np.std(img)/255
    mean_list.append(mean)
    std_list.append(stand_dev)

img_mean = np.mean(mean_list) # loads the mean image value
img_std = np.mean(std_list) # loads the std dev image value

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.62 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
No pneumonia
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
There is an error in the file - test4.dcm. Incorrect body part examined.
Load file test5.dcm ...
There is an error in the file - test5.dcm. Incorrect image type.
Load file test6.dcm ...
There is an error in the file - test6.dcm. Incorrect patient position.
